[View in Colaboratory](https://colab.research.google.com/github/zhh210/flea_market/blob/master/wt_spacy.ipynb)

#spaCy Key Features

Demo of some potentially useful features of spaCy. For a complete list of features, visit [https://spacy.io/usage/linguistic-features](https://spacy.io/usage/linguistic-features)

##Download, Configure, and Initialize

It takes only a few lines to set everything up on Google but configuring a similar environment is overwhelming on MS clusters.

In [1]:
!pip install spacy
!pip install nltk
!python -m spacy download en_core_web_lg

    100% |████████████████████████████████| 22.0MB 1.3MB/s 
    100% |████████████████████████████████| 71kB 20.6MB/s 
    100% |████████████████████████████████| 122kB 25.9MB/s 
    100% |████████████████████████████████| 1.2MB 11.5MB/s 
    100% |████████████████████████████████| 194kB 22.0MB/s 
    100% |████████████████████████████████| 153kB 26.3MB/s 
    100% |████████████████████████████████| 604kB 16.9MB/s 
    100% |████████████████████████████████| 51kB 19.0MB/s 
    100% |████████████████████████████████| 296kB 17.4MB/s 
    100% |████████████████████████████████| 450kB 21.2MB/s 
    100% |████████████████████████████████| 51kB 19.8MB/s 
  Running setup.py bdist_wheel for spacy ... - \ | /

 - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done
  Stored in directory: /content/.cache/pip/wheels/60/0b/bb/7c2e28db574dbb2358176934eddd32a1c5f838ba0bc23eaaab
  Running setup.py bdist_wheel for murmurhash ... - \ | / done
  Stored in directory: /content/.cache/pip/wheels/b8/94/a4/f69f8664cdc1098603df44771b7fec5fd1b3d8364cdd83f512
  Running setup.py bdist_wheel for preshed ... - \ | / - \ | done
  Stored in directory: /content/.cache/pip/wheels/ca/e5/8b/73706d7232da301838e0bc564367a2f7b2fc8f834228fc8a4b
  Running setup.py bdist_wheel for thinc ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done
  Stored in directory: /content/.cache/pip/wh

 \ done
  Stored in directory: /content/.cache/pip/wheels/48/5d/04/22361a593e70d23b1f7746d932802efe1f0e523376a74f321e
Successfully built spacy murmurhash preshed thinc ujson dill regex pathlib cytoolz wrapt
    9% |███                             | 82.0MB 81.7MB/s eta 0:00:10

    100% |████████████████████████████████| 852.3MB 58.2MB/s 
  Running setup.py install for en-core-web-lg ... - \ | / - \ | / - \ | / - \ | / - done

    Linking successful
    /usr/local/lib/python2.7/dist-packages/en_core_web_lg -->
    /usr/local/lib/python2.7/dist-packages/spacy/data/en_core_web_lg

    You can now load the model via spacy.load('en_core_web_lg')



In [0]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_lg')

## Named Entity Recognition
For a complete list of built-in entities, visit [https://spacy.io/api/annotation](https://spacy.io/api/annotation)

In [3]:
displacy.render(nlp(u'''
                   International Business Machines Corp.'s IBM, +0.19% credit rating was downgraded to A1 from A3 Wednesday at Moody's, which predicted Big Blue would continue to be challenged despite making a number of investments to transform into a cloud-services company.
                   '''
                   ),jupyter=True,style='ent')

##Parsing with spaCy

spaCy's powerful at visualization which is very useful for dependency parsing.

In [54]:
displacy.render(nlp(u"Apple's outlook is not very positive."),jupyter=True,style='dep')

In [50]:
displacy.render(nlp(u"Apple's performance is going very strong."),jupyter=True,style='dep')

##Capture Patterned Phrases

You can use `highlight_desc()` to identify and highlight descriptive clauses, i.e. phrases following the patter of: Noun Phrase is Adj.

In [6]:
from spacy.matcher import Matcher
example = u'''
The performance is going very strong. Research Update:
Apple Inc. 'AA+' Rating Affirmed Upon Review Of
Its Financial Policy; Outlook Stable

Overview

 We have reviewed Apple's new financial policy, which is expected to move

the company to a net cash neutral position over time, through declines in
both cash and debt balances, as well as continued improvements to its
overall business through increased scale, consistent profitability, and
strengthened ecosystem through growth of its services business.

 We are affirming all of our ratings on Apple Inc., including the 'AA+'

corporate credit rating.

 The stable outlook reflects our expectation that Apple Inc. will maintain

a commitment to a minimal financial risk profile as it shrinks its
balance sheet over the next several years through shareholder returns,
debt repayments, and potential acquisitions.

Rating Action

On May 23, 2018, S&P Global Ratings affirmed its 'AA+' corporate credit rating
on Cupertino, Calif.-based Apple Inc. The outlook is stable.

At the same time, we are affirming our 'AA+' issue-level rating on the
company's senior unsecured debt and our 'A-1+' rating on its commercial paper.

Rationale

The affirmation of our 'AA+' corporate credit rating reflects Apple's improved
scale and profitability over the past several years and our expectation that
its services segment will provide growing recurring revenue and high margins
despite the competitive smartphone industry landscape, thereby enhancing the
value of its ecosystem and providing more predictable, albeit modest, revenue
growth over the longer term. This, in our view, offsets the revision to its
financial policy whereby Apple intends to become roughly net cash neutral over
time, because we expect the company to maintain credit metrics commensurate
with its minimal financial risk profile and the 'AA+' credit rating.

Apple is the largest U.S.-based provider of mobile devices, personal
computers, and related products and services. The outlook is good. But this one does not count.
'''
def highlight_desc(paragraph=example):
    matcher = Matcher(nlp.vocab)
    matched_sents = [] # collect data of matched sentences to be visualized
    matched_s = []
    
    def collect_sents(matcher, doc, i, matches):
        match_id, start, end = matches[i]
        span = doc[start : end]  # matched span
        sent = span.sent  # sentence containing matched span
        # append mock entity for match in displaCy style to matched_sents
        # get the match span by ofsetting the start and end of the span with the
        # start and end of the sentence in the doc
        match_ents = [{'start': span.start_char - sent.start_char,\
                   'end': span.end_char - sent.start_char,\
                   'label': '[descriptive]'}]
        matched_sents.append({'text': sent.text, 'ents': match_ents })
        
    pattern = [{'POS': 'DET','OP':'?'}, {'POS':'NOUN','OP':'+'}, {'LEMMA': 'be'},{'POS':'VERB','OP':'?'}, {'POS': 'ADV', 'OP': '*'},
           {'POS': 'ADJ'}]
    matcher.add('DESC', collect_sents, pattern)
    
    for sentence in nlp(paragraph).sents:
        matched_sents = []
        matches = matcher(nlp(sentence.text))
        if matched_sents:
            matched_s.append(matched_sents[0])
        else:
            matched_s.append({'text':sentence.text,'ents':[]})
        
    displacy.render(matched_s,manual=True,jupyter=True,style='ent')
  
highlight_desc()

## Pattern Match and Dependency Parsing

In [0]:
import spacy
from spacy.matcher import Matcher

# (empirically derived mean sentiment intensity rating increase for booster words)
B_INCR = 0.293
B_DECR = -0.293

NEGATE = {"aint", "arent", "cannot", "cant", "couldnt", "darent", "didnt", "doesnt",
 "ain't", "aren't", "can't", "couldn't", "daren't", "didn't", "doesn't",
 "dont", "hadnt", "hasnt", "havent", "isnt", "mightnt", "mustnt", "neither",
 "don't", "hadn't", "hasn't", "haven't", "isn't", "mightn't", "mustn't",
 "neednt", "needn't", "never", "none", "nope", "nor", "not", "nothing", "nowhere",
 "oughtnt", "shant", "shouldnt", "uhuh", "wasnt", "werent",
 "oughtn't", "shan't", "shouldn't", "uh-uh", "wasn't", "weren't",
 "without", "wont", "wouldnt", "won't", "wouldn't", "rarely", "seldom", "despite"}

# booster/dampener 'intensifiers' or 'degree adverbs'
# http://en.wiktionary.org/wiki/Category:English_degree_adverbs

BOOSTER_DICT = \
{"absolutely": B_INCR, "amazingly": B_INCR, "awfully": B_INCR, "completely": B_INCR, "considerably": B_INCR,
 "decidedly": B_INCR, "deeply": B_INCR, "effing": B_INCR, "enormously": B_INCR,
 "entirely": B_INCR, "especially": B_INCR, "exceptionally": B_INCR, "extremely": B_INCR,
 "fabulously": B_INCR, "flipping": B_INCR, "flippin": B_INCR,
 "fricking": B_INCR, "frickin": B_INCR, "frigging": B_INCR, "friggin": B_INCR, "fully": B_INCR, "fucking": B_INCR,
 "greatly": B_INCR, "hella": B_INCR, "highly": B_INCR, "hugely": B_INCR, "incredibly": B_INCR,
 "intensely": B_INCR, "majorly": B_INCR, "more": B_INCR, "most": B_INCR, "particularly": B_INCR,
 "purely": B_INCR, "quite": B_INCR, "really": B_INCR, "remarkably": B_INCR,
 "so": B_INCR, "substantially": B_INCR,
 "thoroughly": B_INCR, "totally": B_INCR, "tremendously": B_INCR,
 "uber": B_INCR, "unbelievably": B_INCR, "unusually": B_INCR, "utterly": B_INCR,
 "very": B_INCR,
 "almost": B_DECR, "barely": B_DECR, "hardly": B_DECR, "just enough": B_DECR,
 "kind of": B_DECR, "kinda": B_DECR, "kindof": B_DECR, "kind-of": B_DECR,
 "less": B_DECR, "little": B_DECR, "marginally": B_DECR, "occasionally": B_DECR, "partly": B_DECR,
 "scarcely": B_DECR, "slightly": B_DECR, "somewhat": B_DECR,
 "sort of": B_DECR, "sorta": B_DECR, "sortof": B_DECR, "sort-of": B_DECR}


from spacy.matcher import Matcher, PhraseMatcher
from spacy import displacy

class feature_extractor(Matcher):
    '''Feature extractor and collector, enhancement of spaCy's Matcher'''
    def __init__(self,*kwargs):
        Matcher.__init__(self,*kwargs)
        self._matched = []

    def add(self,match_id,*patterns):
        Matcher.add(self,match_id,self._collect(label=match_id),*patterns)

    def summary(self):
        '''Summarize collected'''
        try:
            for i in self._matched: print(i['label'],self.doc.text[i['start']:i['end']])
            html = displacy.render({'text':self.doc.text,'ents':self._matched},style='ent',manual=True, jupyter=True)
            return html
        except:
            pass
		
	
    def _collect(self,label='desc'):
        '''A callback function to collect matched results of interest'''
        def tmp(matcher,doc,i,matches):
            match_id, start, end = matches[i]

            span = doc[start: end]  # matched span
            sent = span.sent  # sentence containing matched span
            # append mock entity for match in displaCy style to matched_sents
            # get the match span by ofsetting the start and end of the span with the
            # start and end of the sentence in the doc
            match_ents = {'start': span.start_char, 'end': span.end_char, 'label': label, 'index':(start,end)}
            self._matched.append(match_ents)

        return tmp
        
    def __call__(self,*kwargs):
        self._matched = []
        Matcher.__call__(self,*kwargs)
        if isinstance(kwargs[0],spacy.tokens.doc.Doc): self.doc = kwargs[0]

Enhanced matcher for pattern match

In [23]:
# import spacy
# nlp = spacy.load('en_core_web_lg')
fe = feature_extractor(nlp.vocab)
negation_flag = lambda token: token.lower() in NEGATE or "n't" in token.lower()
booster_flag = lambda token: token.lower() in BOOSTER_DICT
IS_NEGATION = nlp.vocab.add_flag(negation_flag)
IS_BOOSTER = nlp.vocab.add_flag(booster_flag)
paragraph = nlp(u"This is a greatly example of negation. This is not an eg of negation. This doesn't count.")

fe.add('BOOST', [{IS_BOOSTER: True}])
fe.add('NEG',[{IS_NEGATION: True}])
matches = fe(paragraph)
print(fe._matched)
fe.summary()

[{'start': 10, 'end': 17, 'index': (3, 4), 'label': 'BOOST'}, {'start': 47, 'end': 50, 'index': (10, 11), 'label': 'NEG'}, {'start': 79, 'end': 82, 'index': (18, 19), 'label': 'NEG'}]
('BOOST', u'greatly')
('NEG', u'not')
('NEG', u"n't")


### Noun Phrase with Dependency Parsing

1) A more complicated pattern to extract noun phrases.

In [87]:
pattern = [{'POS':'NOUN','OP':'+'}, {'LEMMA': 'be'},{'POS':'VERB','OP':'?'}, {'POS': 'ADV', 'OP': '*'},
           {'POS': 'ADJ'}]
np_sentence = nlp(u"Some sentence of not. Apple's performance is not very remarkable. Another sentence.")

fe = feature_extractor(nlp.vocab)
fe.add('NP',pattern)
matches = fe(np_sentence)
fe.summary()

('NP', u'performance is not very remarkable')


2) Dependency parsing to refine the captured pattern

In [88]:
from spacy.symbols import amod, nsubj, nsubjpass, conj, dobj, acomp, advcl, advmod, acl, neg, xcomp, prep, pobj, poss, attr, ccomp, npadvmod, aux, prt, det
res = []
for c in fe._matched:
    sp = np_sentence[c['index'][0]-1:c['index'][1]]
    left = next(sp.root.lefts)
    for right in sp.root.rights:
        
        if right.dep == neg: 
            res.append(right.text)
        elif right.dep == acomp:
            
            chi = next(right.children)
            
            if chi.dep == advmod: res.append(chi.text)
      
    anc = next(left.children)
    res.insert(0,left.text)
    
    if anc.dep == poss: res.insert(0,anc.text)
    
      
print('--'.join(res))

Apple--performance--not--very


In [59]:
displacy.render(np_sentence,style='dep',jupyter=True)

### Verb Phrase with Dependency Parsing

In [76]:
pattern = [{'POS':'NOUN','OP':'+'},{'POS':'VERB','OP':'+'}, {'POS': 'ADV', 'OP': '*'}]
np_sentence = nlp(u"Some sentence of not. Last week, the price dropped sharply in response to the rating change. Another sentence.")

fe = feature_extractor(nlp.vocab)
fe.add('VP',pattern)
matches = fe(np_sentence)
fe.summary()

('VP', u'price dropped sharply')


In [86]:
res = []
for c in fe._matched:
    sp = np_sentence[c['index'][0]-1:c['index'][1]]
    res.append(sp.root.text)
    for left in sp.root.lefts:
        if left.dep == nsubj:
            res.insert(0,left.text)
    for right in sp.root.rights:
        
        if right.dep == advmod:
            res.append(right.text)
      
#     anc = next(left.children)

#     if anc.dep == poss: res.insert(0,anc.text)
    
      
print('--'.join(res))

price--dropped--sharply


In [77]:
displacy.render(np_sentence, style='dep',jupyter=True)